<a href="https://colab.research.google.com/github/afortuny/SustainableFashionAI/blob/main/CircularityAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Analyzing product reviews to understand circularity?

We will leverage the data from https://www.trailrunningreview.com/ , a leading company in product analysis, and we will evalute each trail running shoe from SS22 based on that dimensions:

Circularity:

*   Durability: Is the product make to last?
*   Versatility: can the product be used for multiple conditions /situations?
*   Sustainable materials: is the product made with organic, recycable or vegan materials?

Desirability:

*   Function: Is the product build up appropiate for its purpose?
*   Innovation: Is the product disrupting the market in some sense?
*   Price: Is the product affordable?









In [ ]:
#os.chdir('DeepLearningFastAI/') we will be assuming you are in the folder where the pickle file and brands reside

In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
import fastai
fastai.__version__


Mounted at /content/gdrive


'2.5.6'

In [ ]:
import ipywidgets as widgets
from fastai.vision.all import *
from fastai.vision.widgets import * 

In [ ]:
learn_inf = load_learner('export.pkl')
btn_upload = widgets.FileUpload()
out_pl = widgets.Output()
lbl_pred = widgets.Label()


In [ ]:
def on_data_change(change):
    lbl_pred.value = ''
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'


In [ ]:
btn_upload.observe(on_data_change, names=['data'])


In [ ]:
display(VBox([widgets.Label('Upload your running shoe!'), btn_upload, out_pl, lbl_pred]))